In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import corrcoef, sum, log, arange
import matplotlib
from plotly import tools
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10) # First number width, second number height

In [ ]:
data_AFM = pd.read_excel('../data/Nickelate_mode_amps_SI.xlsx', index_col=0, sheet_name='AFM')
# Drop Overall, Tolerance factor, and Volume columns
foo = ['R1', 'R3', 'R4', 'R5', 'X5', 'M2', 'M3', 'M5', 'Radius', 'TMI', 'TN']
data_AFM = data_AFM[foo]
data_AFM

In [ ]:
data_exp = pd.read_excel('../data/Nickelate_mode_amps_SI.xlsx', index_col=0, sheet_name='Experimental')
data_exp = data_exp[foo]
data_exp

In [ ]:
data_exp_norm = pd.read_excel('../data/Nickelate_mode_amps_SI.xlsx',index_col=0, sheet_name='Experimentalscaled')
data_exp_norm = data_exp_norm[foo]
data_exp_norm

In [ ]:
data_dft_norm = pd.read_excel('../data/Nickelate_mode_amps_SI.xlsx',index_col=0, sheet_name='AFMscaled')
data_dft_norm = data_dft_norm[foo]
data_dft_norm

# Comparison of mean and std of mode amplitudes between experiment and theory

In [ ]:
data1 = data_AFM.copy().round(3)
data2 = data_exp.copy().round(3)

'''
Creates a bar graph of monoclinic mode amplitudes with standard deviation error bars
'''
trace1 = go.Bar(
    x=['R1', 'R3', 'M5'],
    y=data1[['R1', 'R3', 'M5']].mean(axis=0),
    name='Theory',
    error_y=dict(
        type='data',
        array=data1[['R1', 'R3', 'M5']].std(axis=0),
        visible=True
    )
)
trace2 = go.Bar(
    x=['R1', 'R3', 'M5'],
    y=data2[['R1', 'R3', 'M5']].mean(axis=0),
    name='Experiment',
    error_y=dict(
        type='data',
        array=data2[['R1', 'R3', 'M5']].std(axis=0),
        visible=True
    )
)
data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    font=dict(size=36),
    xaxis=dict(title='Mode'),
    yaxis=dict(hoverformat= '.2r', title='Amplitude')
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Nickelates/nickelate_mode_bars')

# Comparison of heatmaps of correlation coefficients

In [ ]:
def heatmap_plot(data, savename=''):
    '''
    Generates a heatmap plot of the mode amplitudes for a given set of data
    '''
    matplotlib.rcParams.update({'font.size': 26})
    values = data.transpose().values
    correlation_matrix = np.corrcoef(values)
    fig, ax = plt.subplots(1, 1)
    plt.pcolor(correlation_matrix, cmap=plt.get_cmap('GnBu'))
    plt.colorbar()
    feature_labels = [r'$R^{+}_{1}$', r'$R^{+}_{3}$', r'$R^{+}_{4}$', r'$R^{+}_{5}$', r'$X^{+}_{5}$', 
                      r'$M^{+}_{2}$', r'$M^{+}_{3}$', r'$M^{+}_{5}$', r'$Radius$', r'$T_{MI}$', r'$T_{N}$']
#     feature_labels=list(data)
    plt.yticks(arange(0.5,len(data.columns)+0.5),feature_labels)
    plt.xticks(arange(0.5,len(data.columns)+0.5),feature_labels)
    fig.tight_layout()
    if savename:
        fig.savefig("../plots/" + savename + '.pdf')
        plt.close(fig)
    else:    
        fig.show()

In [ ]:
heatmap_plot(data_exp, savename='Ni_heat_exp')

In [ ]:
heatmap_plot(data_AFM, savename='Ni_heat_calcAFM')

# Trend of Jahn-Teller with ionic radius

In [ ]:
def plotly_scatter_subplots(data1, data2, var1='R1', var2='R3', var3=None):

    trace1 = go.Scatter(
        x=data1.Radius,
        y=data1[var1],
        mode='lines+markers+text',
        name='Experiment',
        text=data1.index.tolist(),
        textposition='top',
        textfont=dict(
        size=50
        )
    )
    
    trace2 = go.Scatter(
        x=data2.Radius,
        y=data2[var1],
        mode='lines+markers+text',
        name='Theory',
        text=data2.index.tolist(),
        textfont=dict(
        size=50
        ),
        textposition='top'
    )
    
    trace3 = go.Scatter(
        x=data1.Radius,
        y=data1[var2],
        mode='lines+markers+text',
        name='Experiment',
        text=data1.index.tolist(),
        textfont=dict(
        size=50
        ),
        textposition='top'
    )
    trace4 = go.Scatter(
        x=data2.Radius,
        y=data2[var2],
        mode='lines+markers+text',
        name='Theory',
        text=data2.index.tolist(),
        textfont=dict(
        size=50
        ),
        textposition='top'
    )
    if var3:
        trace5 = go.Scatter(
        x=data1.Radius,
        y=data1[var3],
        mode='lines+markers+text',
        name='Experiment',
        text=data1.index.tolist(),
        textfont=dict(
        size=50
            ),
        textposition='top'
        )
        trace6 = go.Scatter(
        x=data2.Radius,
        y=data2[var3],
        mode='lines+markers+text',
        name='Theory',
        text=data2.index.tolist(),
        textfont=dict(
        size=50
            ),
        textposition='top'
        )
        fig = tools.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)
        fig.append_trace(trace1, 1, 1)
        fig.append_trace(trace2, 1, 1)
        fig.append_trace(trace3, 2, 1)
        fig.append_trace(trace4, 2, 1)
        fig.append_trace(trace5, 3, 1)
        fig.append_trace(trace6, 3, 1)
    else:
        fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)
        fig.append_trace(trace1, 1, 1)
        fig.append_trace(trace2, 1, 1)
        fig.append_trace(trace3, 2, 1)
        fig.append_trace(trace4, 2, 1)

#     fig['layout'] = layout
    return fig

In [ ]:
data_exp = pd.read_excel("../data/Nickelate_mode_amps_SI.xlsx", sheet_name='Experimental', index_col=0)
data_AFM = pd.read_excel("../data/Nickelate_mode_amps_SI.xlsx", sheet_name='AFM', index_col=0)
fig = plotly_scatter_subplots(data_exp, data_AFM, var1='X5', var2='Rdisp')
py.iplot(fig, filename='Nickelates/Ni_X5Rdisp_trend')

In [ ]:
fig = plotly_scatter_subplots(data_exp, data_AFM, var1='R3', var2='R1', var3='vol_ratio_poly')
py.iplot(fig, filename='Nickelates/Ni_R3R1polyvol_trend')